# Simple functions

In [1]:
func naiveFib(n int) int {
    if n > 1 {
        return naiveFib(n - 1) + naiveFib(n - 2)
    }
    return 1
}

In [2]:
naiveFib(20)

10946


# Use standard libraries

In [3]:
import (
    "fmt"
    "math"
)

In [4]:
fmt.Printf("sin(pi/3) == %f\n", math.Sin(math.Pi/3))
fmt.Printf("cos(pi/3) == %f\n", math.Cos(math.Pi/3))
fmt.Printf("sqrt(3)/2 == %f\n", math.Sqrt(3)/2)
fmt.Printf("log(e^2) == %f\n", math.Log(math.E * math.E))

sin(pi/3) == 0.866025
cos(pi/3) == 0.500000
sqrt(3)/2 == 0.866025
log(e^2) == 2.000000
21
<nil>


# Non-deterministic code

In [5]:
import (
    "math/rand"
    "time"
)

In [6]:
r := rand.Int()
r

5577006791947779410


In [7]:
r % 1000

410


In [8]:
start := time.Now()
start

2017-10-09 06:24:33.339794746 +0000 UTC m=+4.617622762


In [9]:
end := time.Now()
end

2017-10-09 06:24:33.977383677 +0000 UTC m=+5.255211693


In [10]:
fmt.Printf("end - start = %v", end.Sub(start))

end - start = 637.588931ms26
<nil>


# struct and type

In [11]:
type person struct {
    name string
    age int
}


func (p *person) Hello() string {
    return fmt.Sprintf("Hello! Name: %s, Age: %d", p.name, p.age)
}

In [12]:
p := person{"Alice", 12}
p.Hello()

Hello! Name: Alice, Age: 12


In [13]:
type hello interface {
    Hello() string
}

func printHello(h hello) {
    fmt.Println("h ==", h)
    p, ok := h.(*person)
    if ok {
        fmt.Println("type assertion ok:", p.Hello())
    }
    // This does not work.
    // fmt.Println(h.Hello())
}

p := person{"Alice", 12}
printHello(&p)

h == &{Alice 12}
type assertion ok: Hello! Name: Alice, Age: 12
